In [20]:
import os
from random import randint, random, uniform

import numpy as np
import pandas as pd

import librosa
import matplotlib.pyplot as plt
from tqdm import tqdm
from utils.tdPsola import *

%matplotlib inline

# METHOD='Librosa'
# METHOD = "Morphvox_Pro"
# METHOD = "TD-PSOLA"

# SAMPLE_RATE = 32000
# SAMPLE_RATE = 44100 
# SAMPLE_RATE = 48000
# SAMPLE_RATE = 50000
# SAMPLE_RATE 77= 96000
# SAMPLE_RATE = 8000
SAMPLE_RATE = 9000
# SAMPLE_RATE = 10000
# SAMPLE_RATE = 11000
# SAMPLE_RATE = 12000
# SAMPLE_RATE = 13000





dirName = [f"train{SAMPLE_RATE}", "valid", f"pitchShiftTest{SAMPLE_RATE}"]
MorphvoxPro = range(1,5)
trainFolders: list = sorted(os.listdir(f"./{dirName[0]}/"))
validFolders: list = sorted(os.listdir(f"./{dirName[1]}/"))
pitchShiftFolders: list = sorted(os.listdir(f"./{dirName[2]}/"))
Folders = [trainFolders, validFolders, pitchShiftFolders]

# categorys: list = [f"{i}".rjust(2, "0") for i in range(1, 11)]
DATA: dict = {}
DATA[f"{dirName[0]}"] = []
DATA[f"{dirName[1]}"] = []
DATA[f"{dirName[2]}"] = []
DATA["allFilesList"] = []


categorysCounts: dict = {}
categorysCounts[f"{dirName[0]}"] = {}
categorysCounts[f"{dirName[1]}"] = {}
categorysCounts[f"{dirName[2]}"] = {}


for index, dirn in enumerate(dirName):
    for folder in tqdm(Folders[index]):
        files = os.listdir(f"./{dirn}/" + folder)
        for num, file in enumerate(files):
            DATA["allFilesList"].append(f"./{dirn}/" + folder + "/" + file)
            DATA[f"{dirn}"].append(folder + "/" + file)
            categorysCounts[f"{dirn}"][folder] = num + 1

for i in DATA.keys():
    DATA[f"{i}"] = sorted(DATA[f"{i}"])


def tdpsolaAug(xArray, yArray, path, percent, dataList, fileCounts):
    for index, file in tqdm(enumerate(dataList)):
        file = path + file
        try:
            st: dict = {}
            start = 1
            end = 3
            for i in range(4):
                st[i] = 2 ** (randint(start, end) / 12)
                if i == 3:
                    break
                start += 3
                end += 3
            ps = random() > percent

            xArray[index] = extractFeatures(file)
            yArray[index] = np.int8(file.rsplit("/", 2)[1])
            xArray[fileCounts + index] = extractFeatures(file, ps=ps, st=st[0])
            yArray[fileCounts + index] = np.int8(file.rsplit("/", 2)[1])
            xArray[fileCounts * 2 + index] = extractFeatures(file, ps=ps, st=st[1])
            yArray[fileCounts * 2 + index] = np.int8(file.rsplit("/", 2)[1])
            xArray[fileCounts * 3 + index] = extractFeatures(file, ps=ps, st=st[2])
            yArray[fileCounts * 3 + index] = np.int8(file.rsplit("/", 2)[1])
            xArray[fileCounts * 4 + index] = extractFeatures(file, ps=ps, st=st[3])
            yArray[fileCounts * 4 + index] = np.int8(file.rsplit("/", 2)[1])
        except ValueError:
            print(index, file, ValueError)
    return (xArray, yArray)


def librosaAug(xArray, yArray, path, percent, dataList, fileCounts):
    for index, file in tqdm(enumerate(dataList)):
        file = path + file
        try:
            st = uniform(1.0, 2.0)
            st2 = uniform(2.0, 3.0)
            st3 = uniform(3.0, 5.0)
            st4 = uniform(5.0, 7.0)
            ps = random() > percent

            xArray[index] = extractFeatures(file)
            yArray[index] = np.int8(file.rsplit("/", 2)[1])
            xArray[fileCounts + index] = extractFeatures(file, ps=ps, st=st)
            yArray[fileCounts + index] = np.int8(file.rsplit("/", 2)[1])
            xArray[fileCounts * 2 + index] = extractFeatures(file, ps=ps, st=st2)
            yArray[fileCounts * 2 + index] = np.int8(file.rsplit("/", 2)[1])
            xArray[fileCounts * 3 + index] = extractFeatures(file, ps=ps, st=st3)
            yArray[fileCounts * 3 + index] = np.int8(file.rsplit("/", 2)[1])
            xArray[fileCounts * 4 + index] = extractFeatures(file, ps=ps, st=st4)
            yArray[fileCounts * 4 + index] = np.int8(file.rsplit("/", 2)[1])
        except ValueError:
            print(index, file, ValueError)
    return (xArray, yArray)


def extractFeatures(
    path: str, ps: bool = False, ts: bool = False, st: int = 4
) -> np.ndarray:
    """[提取特徵]
    
    Arguments:
        path {str} -- [路徑]
        ps {bool} 
    Returns:
        np.ndarray -- 
               [
                mfccs,
                mfcc_delta,
                mfcc_delta2,
                chroma,
                mel,
                contrast,
                tonnetz,
                cent,
                flatness,
                rolloff,
                rms,
                ]
    """
    try:
        X, sampleRate = librosa.load(
            path, sr=None, offset=0.0, res_type="kaiser_best", dtype=np.float32,
        )
        if ps:
            if METHOD == "Librosa":
                X = librosa.effects.pitch_shift(X, sampleRate, n_steps=st)

            if METHOD == "TD-PSOLA":
                X = shift_pitch(X, sampleRate, st)
        

        mel = np.mean(librosa.feature.melspectrogram(X, sr=sampleRate).T, axis=0)
        tonnetz = np.mean(
            librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sampleRate).T,
            axis=0,
        )
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sampleRate, n_mfcc=40).T, axis=0)
        mfcc_delta = librosa.feature.delta(mfccs)  # TONY
        mfcc_delta2 = librosa.feature.delta(mfccs, order=2)  # TONY
        stft = np.abs(librosa.stft(X))
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sampleRate).T, axis=0)
        contrast = np.mean(
            librosa.feature.spectral_contrast(S=stft, sr=sampleRate).T, axis=0
        )
        ###### ADD NEW FEATURES (SPECTRAL RELATED)##### 24-SEP
        cent = np.mean(librosa.feature.spectral_centroid(y=X, sr=sampleRate).T, axis=0)
        flatness = np.mean(librosa.feature.spectral_flatness(y=X).T, axis=0)
        rolloff = np.mean(
            librosa.feature.spectral_rolloff(S=stft, sr=sampleRate).T, axis=0
        )
        rms = np.mean(librosa.feature.rms(S=stft).T, axis=0)
        ext_features = np.hstack(
            [
                mfccs,
                mfcc_delta,
                mfcc_delta2,
                chroma,
                mel,
                contrast,
                tonnetz,
                cent,
                flatness,
                rolloff,
                rms,
            ]
        )

    except Exception as e:
        print("Error encountered while parsing file:%s" % (path))
        return None

    return np.array(ext_features)


def creatSets(
    path: str, dataList: list, shape: int, ps: bool = False, st: float = 4
) -> (np.ndarray, np.ndarray):
    """[創建訓練資料]
    
    Arguments:
        path {str} -- [路徑]
        dataList {list} -- [檔案列表]
        shape {tuple} -- [矩陣維度]
    Returns:
        [(np.ndarray, np.ndarray)] -- [(特徵,種類)]
    """
    if METHOD == 'Morphvox_Pro':
        xArray = np.zeros([len(dataList)*4, shape])
        yArray = np.zeros([len(dataList)*4])
        fileCount = len(dataList)
        for num in MorphvoxPro:
            now = fileCount*(num-1)

            for index, file in tqdm(enumerate(dataList)):
                file = path+ f"-{num}/" + file

                try:
                    xArray[now + index] = extractFeatures(file, ps=ps, st=st)
                    yArray[now + index] = file.rsplit("/", 2)[1]
                except ValueError:
                    print(index, file, ValueError)
        return (xArray, yArray)
    else:
        xArray = np.zeros([len(dataList), shape])
        yArray = np.zeros([len(dataList)])

        for index, file in tqdm(enumerate(dataList)):
            file = path + file
            try:
                xArray[index] = extractFeatures(file, ps=ps, st=st)
                yArray[index] = file.rsplit("/", 2)[1]
            except ValueError:
                print(index, file, ValueError)
        return (xArray, yArray)


def creatAugmentSets(
    path: str, dataList: list, shape: int, percent: float = 0
) -> (np.ndarray, np.ndarray):
    """[創建訓練資料]
    
    Arguments:
        path {str} -- [路徑]
        dataList {list} -- [檔案列表]
        shape {tuple} -- [矩陣維度]
    Returns:
        [(np.ndarray, np.ndarray)] -- [(特徵,種類)]
    """
    fileCounts = len(dataList)
    xArray = np.zeros([fileCounts * 5, shape])
    yArray = np.zeros([fileCounts * 5], dtype=np.int8)

    if METHOD == "Librosa":
        return librosaAug(xArray, yArray, path, percent, dataList, fileCounts)

    if METHOD == "TD-PSOLA":
        return tdpsolaAug(xArray, yArray, path, percent, dataList, fileCounts)


def zScore(x):
    return (x - np.mean(x, axis=0)) / np.std(x, axis=0)


100%|██████████| 10/10 [00:00<00:00, 25206.15it/s]


In [20]:
import os
from random import randint, random, uniform

import numpy as np
import pandas as pd

import librosa
import matplotlib.pyplot as plt
from tqdm import tqdm
from utils.tdPsola import *

%matplotlib inline

# METHOD='Librosa'
# METHOD = "Morphvox_Pro"
# METHOD = "TD-PSOLA"

# SAMPLE_RATE = 32000
# SAMPLE_RATE = 44100 
# SAMPLE_RATE = 48000
# SAMPLE_RATE = 50000
# SAMPLE_RATE = 96000
# SAMPLE_RATE = 8000
SAMPLE_RATE = 9000
# SAMPLE_RATE = 10000
# SAMPLE_RATE = 11000
# SAMPLE_RATE = 12000
# SAMPLE_RATE = 13000





dirName = [f"train{SAMPLE_RATE}", "valid", f"pitchShiftTest{SAMPLE_RATE}"]
MorphvoxPro = range(1,5)
trainFolders: list = sorted(os.listdir(f"./{dirName[0]}/"))
validFolders: list = sorted(os.listdir(f"./{dirName[1]}/"))
pitchShiftFolders: list = sorted(os.listdir(f"./{dirName[2]}/"))
Folders = [trainFolders, validFolders, pitchShiftFolders]

# categorys: list = [f"{i}".rjust(2, "0") for i in range(1, 11)]
DATA: dict = {}
DATA[f"{dirName[0]}"] = []
DATA[f"{dirName[1]}"] = []
DATA[f"{dirName[2]}"] = []
DATA["allFilesList"] = []


categorysCounts: dict = {}
categorysCounts[f"{dirName[0]}"] = {}
categorysCounts[f"{dirName[1]}"] = {}
categorysCounts[f"{dirName[2]}"] = {}


for index, dirn in enumerate(dirName):
    for folder in tqdm(Folders[index]):
        files = os.listdir(f"./{dirn}/" + folder)
        for num, file in enumerate(files):
            DATA["allFilesList"].append(f"./{dirn}/" + folder + "/" + file)
            DATA[f"{dirn}"].append(folder + "/" + file)
            categorysCounts[f"{dirn}"][folder] = num + 1

for i in DATA.keys():
    DATA[f"{i}"] = sorted(DATA[f"{i}"])


def tdpsolaAug(xArray, yArray, path, percent, dataList, fileCounts):
    for index, file in tqdm(enumerate(dataList)):
        file = path + file
        try:
            st: dict = {}
            start = 1
            end = 3
            for i in range(4):
                st[i] = 2 ** (randint(start, end) / 12)
                if i == 3:
                    break
                start += 3
                end += 3
            ps = random() > percent

            xArray[index] = extractFeatures(file)
            yArray[index] = np.int8(file.rsplit("/", 2)[1])
            xArray[fileCounts + index] = extractFeatures(file, ps=ps, st=st[0])
            yArray[fileCounts + index] = np.int8(file.rsplit("/", 2)[1])
            xArray[fileCounts * 2 + index] = extractFeatures(file, ps=ps, st=st[1])
            yArray[fileCounts * 2 + index] = np.int8(file.rsplit("/", 2)[1])
            xArray[fileCounts * 3 + index] = extractFeatures(file, ps=ps, st=st[2])
            yArray[fileCounts * 3 + index] = np.int8(file.rsplit("/", 2)[1])
            xArray[fileCounts * 4 + index] = extractFeatures(file, ps=ps, st=st[3])
            yArray[fileCounts * 4 + index] = np.int8(file.rsplit("/", 2)[1])
        except ValueError:
            print(index, file, ValueError)
    return (xArray, yArray)


def librosaAug(xArray, yArray, path, percent, dataList, fileCounts):
    for index, file in tqdm(enumerate(dataList)):
        file = path + file
        try:
            st = uniform(1.0, 2.0)
            st2 = uniform(2.0, 3.0)
            st3 = uniform(3.0, 5.0)
            st4 = uniform(5.0, 7.0)
            ps = random() > percent

            xArray[index] = extractFeatures(file)
            yArray[index] = np.int8(file.rsplit("/", 2)[1])
            xArray[fileCounts + index] = extractFeatures(file, ps=ps, st=st)
            yArray[fileCounts + index] = np.int8(file.rsplit("/", 2)[1])
            xArray[fileCounts * 2 + index] = extractFeatures(file, ps=ps, st=st2)
            yArray[fileCounts * 2 + index] = np.int8(file.rsplit("/", 2)[1])
            xArray[fileCounts * 3 + index] = extractFeatures(file, ps=ps, st=st3)
            yArray[fileCounts * 3 + index] = np.int8(file.rsplit("/", 2)[1])
            xArray[fileCounts * 4 + index] = extractFeatures(file, ps=ps, st=st4)
            yArray[fileCounts * 4 + index] = np.int8(file.rsplit("/", 2)[1])
        except ValueError:
            print(index, file, ValueError)
    return (xArray, yArray)


def extractFeatures(
    path: str, ps: bool = False, ts: bool = False, st: int = 4
) -> np.ndarray:
    """[提取特徵]
    
    Arguments:
        path {str} -- [路徑]
        ps {bool} 
    Returns:
        np.ndarray -- 
               [
                mfccs,
                mfcc_delta,
                mfcc_delta2,
                chroma,
                mel,
                contrast,
                tonnetz,
                cent,
                flatness,
                rolloff,
                rms,
                ]
    """
    try:
        X, sampleRate = librosa.load(
            path, sr=None, offset=0.0, res_type="kaiser_best", dtype=np.float32,
        )
        if ps:
            if METHOD == "Librosa":
                X = librosa.effects.pitch_shift(X, sampleRate, n_steps=st)

            if METHOD == "TD-PSOLA":
                X = shift_pitch(X, sampleRate, st)
        

        mel = np.mean(librosa.feature.melspectrogram(X, sr=sampleRate).T, axis=0)
        tonnetz = np.mean(
            librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sampleRate).T,
            axis=0,
        )
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sampleRate, n_mfcc=40).T, axis=0)
        mfcc_delta = librosa.feature.delta(mfccs)  # TONY
        mfcc_delta2 = librosa.feature.delta(mfccs, order=2)  # TONY
        stft = np.abs(librosa.stft(X))
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sampleRate).T, axis=0)
        contrast = np.mean(
            librosa.feature.spectral_contrast(S=stft, sr=sampleRate).T, axis=0
        )
        ###### ADD NEW FEATURES (SPECTRAL RELATED)##### 24-SEP
        cent = np.mean(librosa.feature.spectral_centroid(y=X, sr=sampleRate).T, axis=0)
        flatness = np.mean(librosa.feature.spectral_flatness(y=X).T, axis=0)
        rolloff = np.mean(
            librosa.feature.spectral_rolloff(S=stft, sr=sampleRate).T, axis=0
        )
        rms = np.mean(librosa.feature.rms(S=stft).T, axis=0)
        ext_features = np.hstack(
            [
                mfccs,
                mfcc_delta,
                mfcc_delta2,
                chroma,
                mel,
                contrast,
                tonnetz,
                cent,
                flatness,
                rolloff,
                rms,
            ]
        )

    except Exception as e:
        print("Error encountered while parsing file:%s" % (path))
        return None

    return np.array(ext_features)


def creatSets(
    path: str, dataList: list, shape: int, ps: bool = False, st: float = 4
) -> (np.ndarray, np.ndarray):
    """[創建訓練資料]
    
    Arguments:
        path {str} -- [路徑]
        dataList {list} -- [檔案列表]
        shape {tuple} -- [矩陣維度]
    Returns:
        [(np.ndarray, np.ndarray)] -- [(特徵,種類)]
    """
    if METHOD == 'Morphvox_Pro':
        xArray = np.zeros([len(dataList)*4, shape])
        yArray = np.zeros([len(dataList)*4])
        fileCount = len(dataList)
        for num in MorphvoxPro:
            now = fileCount*(num-1)

            for index, file in tqdm(enumerate(dataList)):
                file = path+ f"-{num}/" + file

                try:
                    xArray[now + index] = extractFeatures(file, ps=ps, st=st)
                    yArray[now + index] = file.rsplit("/", 2)[1]
                except ValueError:
                    print(index, file, ValueError)
        return (xArray, yArray)
    else:
        xArray = np.zeros([len(dataList), shape])
        yArray = np.zeros([len(dataList)])

        for index, file in tqdm(enumerate(dataList)):
            file = path + file
            try:
                xArray[index] = extractFeatures(file, ps=ps, st=st)
                yArray[index] = file.rsplit("/", 2)[1]
            except ValueError:
                print(index, file, ValueError)
        return (xArray, yArray)


def creatAugmentSets(
    path: str, dataList: list, shape: int, percent: float = 0
) -> (np.ndarray, np.ndarray):
    """[創建訓練資料]
    
    Arguments:
        path {str} -- [路徑]
        dataList {list} -- [檔案列表]
        shape {tuple} -- [矩陣維度]
    Returns:
        [(np.ndarray, np.ndarray)] -- [(特徵,種類)]
    """
    fileCounts = len(dataList)
    xArray = np.zeros([fileCounts * 5, shape])
    yArray = np.zeros([fileCounts * 5], dtype=np.int8)

    if METHOD == "Librosa":
        return librosaAug(xArray, yArray, path, percent, dataList, fileCounts)

    if METHOD == "TD-PSOLA":
        return tdpsolaAug(xArray, yArray, path, percent, dataList, fileCounts)


def zScore(x):
    return (x - np.mean(x, axis=0)) / np.std(x, axis=0)


100%|██████████| 10/10 [00:00<00:00, 25206.15it/s]


In [ ]:
train = DATA[f'{dirName[0]}']
valid = DATA[f'{dirName[1]}']
test = DATA[f'{dirName[2]}']

if METHOD == 'Morphvox_Pro':
    trainData, trainLabel = creatSets(
    f"./{dirName[0]}", train, extractFeatures(f"./{dirName[0]}/" + train[0]).shape[0], False
)
    pitchShiftTestData, pitchShiftTestLabel = creatSets(
        f"./{dirName[2]}",
        test,
        extractFeatures(f"./{dirName[2]}/" + test[3]).shape[0],
        False
    )
    
else :    
    trainData, trainLabel = creatAugmentSets(
        f"./{dirName[0]}/", train, extractFeatures(f"./{dirName[0]}/" + train[0]).shape[0], 0
    )
    validData, validLabel = creatSets(
        f"./{dirName[1]}/", valid, extractFeatures(f"./{dirName[1]}/" + valid[0]).shape[0], False
    )
    pitchShiftTestData, pitchShiftTestLabel = creatSets(
        f"./{dirName[2]}/",
        test,
        extractFeatures(f"./{dirName[2]}/" + test[3]).shape[0],
        True,
        3,
    )

# if METHOD == 'Morphvox_Pro':
#     pass
# else:
#     validData, validLabel = creatSets(
#         f"./{dirName[1]}/", valid, extractFeatures(f"./{dirName[1]}/" + valid[0]).shape[0], False
#     )
    
# if METHOD == 'Morphvox_Pro':
#     pitchShiftTestData, pitchShiftTestLabel = creatSets(
#         f"./{dirName[2]}/",
#         test,
#         extractFeatures(f"./{dirName[2]}/" + test[3]).shape[0],
#          False
#     )
# else:
#     pitchShiftTestData, pitchShiftTestLabel = creatSets(
#         f"./{dirName[2]}/",
#         test,
#         extractFeatures(f"./{dirName[2]}/" + test[3]).shape[0],
#         True,
#         3,
#     )



17it [01:50,  6.16s/it]

# 儲存資料
- 訓練資料
- 驗證資料

In [ ]:
indices = np.random.permutation(trainData.shape[0])
trainData = trainData[indices]

trainLabel = trainLabel[indices]

# indices = np.random.permutation(validData.shape[0])
# validData = validData[indices]
# validLabel = validLabel[indices]

indices = np.random.permutation(pitchShiftTestData.shape[0])
pitchShiftTestData = pitchShiftTestData[indices]
pitchShiftTestLabel = pitchShiftTestLabel[indices]

np.save(f"./pre-train/{METHOD}/Data-{dirName[0]}-{SAMPLE_RATE}.npy", zScore(trainData))
np.save(f"./pre-train/{METHOD}/Label-{dirName[0]}-{SAMPLE_RATE}.npy", trainLabel.astype(np.int))

# np.save(f"./pre-valid/{METHOD}/Data-{dirName[1]}.npy", zScore(validData))
# np.save(f"./pre-valid/{METHOD}/Label-{dirName[1]}.npy", validLabel.astype(np.int))

np.save(f"./pre-test/{METHOD}/Data-{dirName[2]}-{SAMPLE_RATE}.npy", zScore(pitchShiftTestData))
np.save(f"./pre-test/{METHOD}/Label-{dirName[2]}-{SAMPLE_RATE}.npy", pitchShiftTestLabel.astype(np.int))

In [ ]:
# def extractMelSpec(
#     path: str, flip: bool = False, ps: bool = False, st: int = 4
# ) -> np.ndarray:
#     """[提取mel頻譜]

#     Arguments:
#         path {str} -- [路徑]

#     Keyword Arguments:
#         flip {bool} -- [矩陣反轉] (default: {False})
#         ps {bool} -- [是否調整音階]] (default: {False})
#         st {int} -- [調整幾階]] (default: {4})

#     Returns:
#         np.ndarray -- [mel頻譜]
#     """
#     try:
#         sig, rate = librosa.load(
#             path, offset=0.0, res_type="kaiser_fast", dtype=np.float32
#         )
#         if len(sig) < 22050:  # pad shorter than 1 sec audio with ramp to zero
#             sig = np.pad(sig, (0, 22050 - len(sig)), "linear_ramp")
#         if ps:
#             sig = librosa.effects.pitch_shift(sig, rate, n_steps=st)
#         db = librosa.amplitude_to_db(
#             librosa.stft(sig[:22050], hop_length=256, center=False), ref=np.max
#         )
#         spec = librosa.feature.melspectrogram(S=db, n_mels=128).T
#         if flip:
#             spec = np.flipud(spec)
#     except Exception as e:
#         print("Error encountered while parsing file:%s" % (path))
#         return None
#     return spec.astype(np.float32)



# def creatMelSpecSets(
#     path: str, dataList: list, shape: tuple
# ) -> (np.ndarray, np.ndarray):
#     """[創建訓練梅爾頻譜資料]

#     Arguments:
#         path {str} -- [路徑]
#         dataList {list} -- [檔案列表]
#         shape {tuple} -- [矩陣維度]
#     Returns:
#         [(np.ndarray, np.ndarray)] -- [(特徵,種類)]
#     """
#     shape1, shape2 = shape
#     xArray = np.zeros([len(dataList), shape1, shape2])
#     yArray = np.zeros([len(dataList)])
#     for index, file in enumerate(dataList):
#         file = path + file
#         try:
#             xArray[index] = extractMelSpec(file)
#             yArray[index] = file.rsplit("/", 2)[1]
#         except ValueError:
#             print(index, file, ValueError)
#     return (xArray, yArray)




# melTrainData, melTrainLabel = creatMelSpecSets(
#     "./train/", trainList, extractMelSpec("./train/" + trainList[0]).shape
# )

# melValidData, melValidLabel = creatMelSpecSets(
#     "./valid/", validList, extractMelSpec("./valid/" + validList[0]).shape
# )

In [31]:
#合併
for index ,data in enumerate(["Librosa","Morphvox_Pro","TD-PSOLA"]):
    temp1 = np.load(f"./pre-train/{data}/Data-train-{SAMPLE_RATE}.npy", allow_pickle=True)
    temp2 = np.load(f"./pre-train/{data}/Label-train-{SAMPLE_RATE}.npy", allow_pickle=True)
    temp3 = np.load(f"./pre-test/{data}/Data-pitchShiftTest-{SAMPLE_RATE}.npy", allow_pickle=True)
    temp4 = np.load(f"./pre-test/{data}/Label-pitchShiftTest-{SAMPLE_RATE}.npy", allow_pickle=True)
    if index == 0 :
        trainData = temp1
        trainLabel = temp2
        testData = temp3
        testLabel = temp4
        continue
    trainData = np.concatenate((trainData, temp1), axis=0)
    trainLabel = np.concatenate((trainLabel, temp2), axis=0)
    testData = np.concatenate((testData, temp3), axis=0)
    testLabel = np.concatenate((testLabel, temp4), axis=0)
#打亂   
indices = np.random.permutation(trainData.shape[0])
trainData = trainData[indices]
trainLabel = trainLabel[indices]
indices = np.random.permutation(testData.shape[0])
testData = testData[indices]
testLabel = testLabel[indices]
#儲存
np.save(f"./pre-train/Data-All-{SAMPLE_RATE}.npy", trainData)
np.save(f"./pre-train/Label-All-{SAMPLE_RATE}.npy", trainLabel.astype(np.int))
np.save(f"./pre-test/Data-All-{SAMPLE_RATE}.npy", testData)
np.save(f"./pre-test/Label-All-{SAMPLE_RATE}.npy", testLabel.astype(np.int))


In [3]:
indices = np.random.permutation(trainData.shape[0])
trainData = trainData[indices]
trainLabel = trainLabel[indices]
indices = np.random.permutation(pitchShiftTestData.shape[0])
testData = testData[indices]
testLabel = testLabel[indices]

In [28]:
np.save(f"./pre-train/Data-All.npy", trainData)
np.save(f"./pre-train/Label-All.npy", trainLabel.astype(np.int))
np.save(f"./pre-test/Data-All.npy", testData)
np.save(f"./pre-test/Label-All.npy", testLabel.astype(np.int))

In [5]:
trainLable.shape

(2800,)

In [4]:
trainLabel 

array([3, 6, 6, ..., 1, 6, 8])

In [17]:
from IPython.display import Audio

X, sampleRate = librosa.load(
            "./01-20-Copy1.wav", sr=None,offset=0.0, res_type="kaiser_best",
        )

X2, sampleRate2 =librosa.load(
            "./01-20-Copy2.wav", sr=96000, offset=0.0, res_type="kaiser_best", dtype=np.float32,
        )

Audio(X,rate=sampleRate)

In [18]:
Audio(X2, rate=sampleRate2)